In [1]:
!pip install awswrangler
!pip install pandas
!pip install sklearn

import awswrangler as wr
import pandas as pd
import sklearn as sk
import matplotlib
import matplotlib.pyplot as plt
import math

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p38/bin/python -m pip install --upgrade pip' command.
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p38/bin/python -m pip install --upgrade pip' command.
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p38/bin/python -m pip install --upgrade pip' command.


In [2]:
print(wr.__version__)
print(pd.__version__)
print(sk.__version__)
print(matplotlib.__version__)

2.16.1
1.3.4
1.0.1
3.5.0


In [3]:
input_path = "s3://staging-clean-wateringsensors-123/v2.csv"
raw_data = wr.s3.read_csv(input_path)
raw_data

,timestamp,dist,depth,value,hour,date,month,year,sensor
0,1626307200,0.0,-60.0,-27.366967,2021-07-15 00,2021-07-15,2021-07,2021,"(0.0, -60.0)"
1,1626307200,0.0,-40.0,-21.041212,2021-07-15 00,2021-07-15,2021-07,2021,"(0.0, -40.0)"
2,1626307200,0.0,-20.0,-21.041212,2021-07-15 00,2021-07-15,2021-07,2021,"(0.0, -20.0)"
3,1626307200,25.0,-60.0,-193.593787,2021-07-15 00,2021-07-15,2021-07,2021,"(25.0, -60.0)"
4,1626307200,25.0,-40.0,-23.102383,2021-07-15 00,2021-07-15,2021-07,2021,"(25.0, -40.0)"
...,...,...,...,...,...,...,...,...,...
8751,1628982000,50.0,-40.0,-840.353649,2021-08-14 23,2021-08-14,2021-08,2021,"(50.0, -40.0)"
8752,1628982000,50.0,-20.0,-129.913714,2021-08-14 23,2021-08-14,2021-08,2021,"(50.0, -20.0)"
8753,1628982000,80.0,-60.0,-462.190096,2021-08-14 23,2021-08-14,2021-08,2021,"(80.0, -60.0)"
8754,1628982000,80.0,-40.0,-791.964873,2021-08-14 23,2021-08-14,2021-08,2021,"(80.0, -40.0)"


In [4]:
time_hierarchy = raw_data[["timestamp", "date", "month", "year"]].drop_duplicates()
time_hierarchy

,timestamp,date,month,year
0,1626307200,2021-07-15,2021-07,2021
12,1626310800,2021-07-15,2021-07,2021
24,1626314400,2021-07-15,2021-07,2021
36,1626318000,2021-07-15,2021-07,2021
48,1626321600,2021-07-15,2021-07,2021
...,...,...,...,...
8696,1628967600,2021-08-14,2021-08,2021
8708,1628971200,2021-08-14,2021-08,2021
8720,1628974800,2021-08-14,2021-08,2021
8732,1628978400,2021-08-14,2021-08,2021


In [5]:
sensor_hierarchy = raw_data[["sensor", "dist", "depth"]].drop_duplicates()
sensor_hierarchy

,sensor,dist,depth
0,"(0.0, -60.0)",0.0,-60.0
1,"(0.0, -40.0)",0.0,-40.0
2,"(0.0, -20.0)",0.0,-20.0
3,"(25.0, -60.0)",25.0,-60.0
4,"(25.0, -40.0)",25.0,-40.0
5,"(25.0, -20.0)",25.0,-20.0
6,"(50.0, -60.0)",50.0,-60.0
7,"(50.0, -40.0)",50.0,-40.0
8,"(50.0, -20.0)",50.0,-20.0
9,"(80.0, -60.0)",80.0,-60.0


In [6]:
measure_fact = raw_data[["sensor", "timestamp", "value"]].drop_duplicates()
measure_fact

,sensor,timestamp,value
0,"(0.0, -60.0)",1626307200,-27.366967
1,"(0.0, -40.0)",1626307200,-21.041212
2,"(0.0, -20.0)",1626307200,-21.041212
3,"(25.0, -60.0)",1626307200,-193.593787
4,"(25.0, -40.0)",1626307200,-23.102383
...,...,...,...
8751,"(50.0, -40.0)",1628982000,-840.353649
8752,"(50.0, -20.0)",1628982000,-129.913714
8753,"(80.0, -60.0)",1628982000,-462.190096
8754,"(80.0, -40.0)",1628982000,-791.964873


In [ ]:
host = "---"
port = 5432
user = "postgres"
pwd = "bigdata2022"
db = "sensordwh"

from sqlalchemy import create_engine
s = 'postgresql://{}:{}@{}:{}/{}'.format(user, pwd, host, str(port), db)
engine = create_engine(s)

In [8]:
sensor_hierarchy.to_sql('sensor', engine, index=False, if_exists='replace')

In [9]:
time_hierarchy.to_sql('date', engine, index=False, if_exists='replace')

In [10]:
measure_fact.to_sql('measurement', engine, index=False, if_exists='replace')